In [1]:
import sys
import subprocess
import os 

import glob
import multiprocessing

In [ ]:
sys.path.append('../../')
from extract_glycan_info import parse_gf_result

In [ ]:
phi_dihedral=['C2','C1','O2','C2']
psi_dihedral=['C1','O2','C2','C3']

di_sugar=['GLC', 'GLC']
patterns1= ['GLC(*_*_)]-GLC(*_*_)', 'GLC(*_*_)-GLC(*_*_)','GLC(*_*_)]-[*]-GLC(*_*_)', 'GLC(*_*_)-[*]-GLC(*_*_)']
patterns2= ['DGlcpa1-2DGlcpa', 'DGlcpa1-2]DGlcpa', 'DGlcpa1-2[*]DGlcpa']
patterns=(patterns1,patterns2)

gf_result ='../../data_gf_query/a1-2D_GF_results.csv'

chimerax_path= '/bin/chimerax'
bfmp_path = '/home/rajan/apps/BFMP/detect_shape'


instant1 = parse_gf_result(gf_result, phi_dihedral, psi_dihedral, di_sugar, patterns, chimerax_path, bfmp_path)
instant1.read_gf_file()
instant1.dnl_pdbs()
instant1.create_pdb_glycan_dict() 
instant1.glycan_count()
instant1.select_pdbs()

In [4]:
%%capture output
instant1.generate_chimerax_script_to_compute_dihedral()

In [5]:
# Write the entire output to a file
with open('calc_dihedral2.cxc', 'w') as f:
    f.write(output.stdout)

In [ ]:
# Use the custom split function
instant1.custom_split_chimerax_script('calc_dihedral2.cxc', 'calc_dihedral_part_')

In [ ]:
# Run the parallel execution and stitching
instant1.run_chimerax_script_parallel('./calc_dihedral_chimerax/calc_dihedral_part_*.cxc')

In [8]:
# instant1.run_chimerax_script('calc_dihedral2.cxc', 'calc_dihedral2.out')

In [ ]:
# torsions = instant1.clean_up_chimerax_output('calc_dihedral2.out')
torsions = instant1.clean_up_chimerax_output('combined_chimerax_output.out')
torsions.to_csv('chimerax_output.csv', index=False)

In [10]:
instant1.calc_BFactor()

In [ ]:
instant1.find_bfmp_ring_shape()

In [ ]:
output_df = instant1.match_glycan_tree()

In [13]:
output_df.to_csv('DGlcpa1-2DGlcpa_ver2.1.csv',sep=',',index=False, columns=['pdb','glycam name','glycan','phi site','psi site','phi','psi','B factor(Avg)','BFMP ring shape'])
output_df.to_csv('DGlcpa1-2DGlcpa_ver3.csv',sep=',',index=False, columns=['pdb','phi site','psi site','phi','psi','B factor(Avg)','BFMP ring shape'])

In [14]:
################################
# print(instant1.pdbs)
# print(instant1.pdb_glycan_dict)

In [ ]:
print(instant1.torsions)

In [15]:
instant1.plot(torsions=instant1.torsions, title='DGlcpa1-2DGlcpa',max_histogram_scale=100,step_histogram_tick=20)